In [1]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión de CUDA:", torch.version.cuda)
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectada")
print("CUDA disponible:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
torch.backends.cudnn.benchmark = True  # Optimiza el uso de la GPU
torch.cuda.empty_cache()  # Libera memoria de GPU antes de entrenar
print("Versión de PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("Versión de CUDA en PyTorch:", torch.version.cuda)
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectada")

CUDA disponible: True
Versión de CUDA: 12.1
Nombre de la GPU: NVIDIA GeForce RTX 4090
CUDA disponible: True
Número de GPUs: 1
Versión de PyTorch: 2.5.1
CUDA disponible: True
Versión de CUDA en PyTorch: 12.1
Nombre de la GPU: NVIDIA GeForce RTX 4090


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Transformaciones para normalizar imágenes (Requeridas por ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar imágenes
    transforms.ToTensor(),  # Convertir a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización de ResNet
])
# Rutas a los conjuntos de entrenamiento y validación
train_dir = "./Data/train"
valid_dir = "./Data/valid"
# Cargar imágenes desde carpetas (PyTorch usa estructura basada en subcarpetas para cada clase)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
# Definir DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
# Verificar categorías detectadas
print("Clases detectadas:", train_dataset.classes)
# Definir modelo y modificar la última capa
num_classes = 6  # Número de categorías
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Ajustar capa final
model = model.to(device)
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Entrenamiento del modelo
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.2f}%, Time: {time.time() - start_time:.2f}s")
# Evaluación del modelo en validación
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
val_acc = 100 * correct / total
print(f"Accuracy en validación: {val_acc:.2f}%")
# Guardar el modelo entrenado
torch.save(model.state_dict(), "Modelo_MadurezPlatano.pth")
print("Modelo guardado como 'Modelo_MadurezPlatano.pth'")

Clases detectadas: ['freshripe', 'freshunripe', 'overripe', 'ripe', 'rotten', 'unripe']
Epoch 1/20, Loss: 0.2959, Acc: 90.25%, Time: 34.59s
Epoch 2/20, Loss: 0.1438, Acc: 94.93%, Time: 31.37s
Epoch 3/20, Loss: 0.0914, Acc: 97.24%, Time: 25.25s
Epoch 4/20, Loss: 0.1115, Acc: 96.33%, Time: 26.30s
Epoch 5/20, Loss: 0.1071, Acc: 96.35%, Time: 25.43s
Epoch 6/20, Loss: 0.0622, Acc: 98.11%, Time: 25.38s
Epoch 7/20, Loss: 0.0523, Acc: 98.53%, Time: 30.78s
Epoch 8/20, Loss: 0.0872, Acc: 96.84%, Time: 32.19s
Epoch 9/20, Loss: 0.0483, Acc: 98.29%, Time: 29.06s
Epoch 10/20, Loss: 0.0431, Acc: 98.78%, Time: 31.03s
Epoch 11/20, Loss: 0.0590, Acc: 98.13%, Time: 31.37s
Epoch 12/20, Loss: 0.0265, Acc: 99.11%, Time: 29.72s
Epoch 13/20, Loss: 0.0262, Acc: 99.24%, Time: 31.54s
Epoch 14/20, Loss: 0.0451, Acc: 98.78%, Time: 31.47s
Epoch 15/20, Loss: 0.0602, Acc: 98.44%, Time: 29.59s
Epoch 16/20, Loss: 0.0602, Acc: 98.06%, Time: 28.67s
Epoch 17/20, Loss: 0.0240, Acc: 99.38%, Time: 31.21s
Epoch 18/20, Loss: 0